# DRAGON relationships pre-processing

See [Makefile](Makefile) for how the results were generated

In [1]:
# initial imports
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from oaklib import get_adapter
from oaklib.interfaces.semsim_interface import SemanticSimilarityInterface

from typing import List, Dict

from pathlib import Path

import yaml
import glob

In [9]:
def connect(ont: str) -> pd.DataFrame:
    """
    Connect to an ontology via OAK/sqlite
    
    Assumes already in db folder
    :param ont: 
    :return: 
    """
    return get_adapter(f"downloads/{ont}.db")

cl = connect("cl")

INFO:root:No schema: assuming file path downloads/cl.db
INFO:root:Locator: sqlite:////Users/cjm/repos/dragon-ai-results/downloads/cl.db
INFO:root:Locator, post-processed: sqlite:////Users/cjm/repos/dragon-ai-results/downloads/cl.db


In [3]:
from linkml_runtime.utils.formatutils import camelcase
from collections import defaultdict
from functools import lru_cache


@lru_cache
def labels2ids(ont: str) -> Dict[str, List[str]]:
    """
    Get a map of (camelcase) labels to ids for an ontology.

    This is to reverse-map the results which are in CamelCase form
    
    :param ont: e.g. cl
    :return: 
    """
    m = defaultdict(list)
    adapter = connect(ont)
    for id, lbl in adapter.labels(list(adapter.entities())):
        if not lbl:
            continue
        lbl = camelcase(lbl)
        m[lbl].append(id)
    return m

cl_map = labels2ids("cl")
len(cl_map)

INFO:root:No schema: assuming file path downloads/cl.db
INFO:root:Locator: sqlite:////Users/cjm/repos/dragon-ai-results/downloads/cl.db
INFO:root:Locator, post-processed: sqlite:////Users/cjm/repos/dragon-ai-results/downloads/cl.db
INFO:root:Query: SELECT node.id AS node_id 
FROM node 
WHERE (node.id NOT IN (SELECT deprecated_node.id 
FROM deprecated_node))


16012

In [4]:
def revmap(m: Dict[str, List[str]]) -> Dict[str, str]:
    """
    Reverse a one to many map into a one to one map
    
    :param m: 
    :return: 
    """
    r = {}
    for k, v in m.items():
        for x in v:
            r[x] = k
    return r

In [57]:

from oaklib.interfaces import OboGraphInterface
from oaklib.datamodels.vocabulary import IS_A, PART_OF
from oaklib.utilities.obograph_utils import graph_to_image, default_stylemap_path

from copy import copy
from pydantic import BaseModel

In [58]:
from pydantic import ConfigDict
from enum import Enum
from typing import Optional


class OutcomeType(str, Enum):
    TRUE_POSITIVE = "true_positive"
    FALSE_POSITIVE = "false_positive"
    FALSE_NEGATIVE = "false_negative"
    NULL = "null"
    
    
class QualifierType(str, Enum):
    PREDICTION_IS_MORE_GENERAL = "prediction_is_more_general"
    PREDICTION_IS_MORE_SPECIFIC = "prediction_is_more_specific"
    

class RelationshipEval(BaseModel):
    #model_config = ConfigDict(use_enum_values=True)
    ontology: Optional[str] = None
    method: Optional[str] = None
    term_id: str
    term_label: Optional[str] = None
    pred: Optional[str] = None
    outcome: Optional[OutcomeType] = None
    qualifier: Optional[QualifierType] = None
    expected_tgt: Optional[str] = None
    predicted_tgt: Optional[str] = None
    predicted_tgt_exists: bool = True
    ic_ratio: Optional[float] = None
    
    def destrictify(self):
        if self.qualifier:
            self.outcome = OutcomeType.NULL
            
    def as_dict(self):
        obj = self.model_dump()
        obj["outcome"] = self.outcome.value
        obj["qualifier"] = self.qualifier.value if self.qualifier else None
        return obj
            

t = RelationshipEval(term_id="foo", outcome = OutcomeType.TRUE_POSITIVE)
print(t.as_dict())

{'ontology': None, 'method': None, 'term_id': 'foo', 'term_label': None, 'pred': None, 'outcome': 'true_positive', 'qualifier': None, 'expected_tgt': None, 'predicted_tgt': None, 'predicted_tgt_exists': True, 'ic_ratio': None}


In [79]:
from oaklib.interfaces import OboGraphInterface
from oaklib.datamodels.vocabulary import IS_A, PART_OF
from oaklib.utilities.obograph_utils import graph_to_image, default_stylemap_path

def get_relationship_outcomes(expected_rels, predicted_rels, ont, term_id:str) -> List[RelationshipEval]:
    """
    Score predicted relationships against expected.
    """
    labelmap = labels2ids(ont)
    idmap = revmap(labelmap)
    new_terms = set()
    adapter = get_adapter(f"sqlite:obo:{ont}")
    if not isinstance(adapter, OboGraphInterface):
        raise Exception(f"Only OboGraphInterface supported")
    all_preds = set(expected_rels.keys()).union(set(predicted_rels.keys()))
    all_predicted_tgts = set()
    rows = []
    for pred in all_preds:
        # print(pred)
        if pred.lower() == 'subclassof':
            preds = [IS_A]
        else:
            pred_ids = labelmap.get(pred)
            if not pred_ids:
                pred_ids = [ids[0] for lbl, ids in labelmap.items() if pred.lower() == lbl.lower()]
            if not pred_ids:
                print(f"MISSING: {pred}")
                continue
                preds = [IS_A]
            else:
                preds = [IS_A, *pred_ids]
        preds = list(set(preds))
        def lbl2id(lbl):
            return labelmap.get(lbl, [lbl])
        expected_tgts = {x: lbl2id(x) for x in expected_rels.get(pred, [])}
        predicted_tgts = {x: lbl2id(x) for x in predicted_rels.get(pred, [])}
        for xs in predicted_tgts.values():
            all_predicted_tgts.update(xs)
        new_terms.update({x for x in predicted_tgts.keys() if x not in labelmap})
        #print(f"PRED: {pred} // predicted= {predicted_tgts} // expected= {expected_tgts}")
        
        predicted_accounted_for = set()
        expected_accounted_for = set()
        for expected_tgt in expected_tgts:
            row = RelationshipEval(term_id=term_id,
                                   term_label=adapter.label(term_id),
                                   pred=pred,
                                   expected_tgt=expected_tgt)
            if expected_tgt in predicted_tgts:
                #print(f"TP: {pred} {expected_tgt}")
                row.outcome = OutcomeType.TRUE_POSITIVE
                row.predicted_tgt = expected_tgt
                predicted_accounted_for.add(expected_tgt)
                expected_accounted_for.add(expected_tgt)
                rows.append(row)
            else:
                for expected_tgt_id in expected_tgts.get(expected_tgt, []):
                    ancs = adapter.ancestors(expected_tgt_id, preds)
                    ancs_as_lbls = [idmap.get(x,x) for x in ancs]
                    more_general_predictions = [x for x in ancs_as_lbls if x in predicted_tgts]
                    # note: we check for more specific later
                    if more_general_predictions:
                        #print(f"n/a PREDICTED_IS_MORE_GENERAL: {pred} {expected_tgt} // more_general_preds: {more_general_predictions}")
                        predicted_accounted_for.update(more_general_predictions)
                        expected_accounted_for.add(expected_tgt)
                        row.outcome = OutcomeType.FALSE_NEGATIVE
                        row.qualifier = QualifierType.PREDICTION_IS_MORE_GENERAL
                        row.predicted_tgt = "; ".join(more_general_predictions)
                        if more_general_predictions:
                            predicted_tgt_id = lbl2id(more_general_predictions[0])[0]
                            if not isinstance(adapter, SemanticSimilarityInterface):
                                raise Exception(f"Only SemanticSimilarityInterface supported")
                            parent_ic = adapter.get_information_content(predicted_tgt_id, predicates=[IS_A, PART_OF])
                            child_ic = adapter.get_information_content(expected_tgt_id, predicates=[IS_A, PART_OF])
                            row.ic_ratio = parent_ic / child_ic if (child_ic and parent_ic) else 0.5 
                        rows.append(row)
                        break
        for predicted_tgt in predicted_tgts:
            row = RelationshipEval(term_id=term_id,
                                   term_label=adapter.label(term_id),
                                   pred=pred,
                                   predicted_tgt=predicted_tgt,
                                   )
            if predicted_tgt in predicted_accounted_for:
                continue
            if predicted_tgt not in labelmap:
                row.predicted_tgt_exists = False
                #if ignore_ungrounded:
                #    continue
            ancs = adapter.ancestors(term_id, preds)
            ancs_as_lbls = [idmap.get(x, x) for x in ancs]
            #print(f"CHECKING {term_id} {pred} {predicted_tgt} // ancs: {ancs_as_lbls}")
            if predicted_tgt in ancs_as_lbls:
                row.outcome = OutcomeType.TRUE_POSITIVE
                continue
            row.outcome = OutcomeType.FALSE_POSITIVE
            for predicted_tgt_id in predicted_tgts.get(predicted_tgt, []):
                ancs = adapter.ancestors(predicted_tgt_id, preds)
                ancs_as_lbls = [idmap.get(x, x) for x in ancs]
                #print(pred, predicted_tgt_id, ancs_as_lbls)
                more_general_expected = [x for x in ancs_as_lbls if x in expected_tgts]
                if more_general_expected:
                    #print(f"?? PREDICTED_IS_MORE_SPECIFIC: {pred} {expected_tgt} // more_general_expected: {more_general_expected}")
                    expected_accounted_for.update(more_general_expected)
                    row.outcome = OutcomeType.FALSE_POSITIVE
                    row.qualifier = QualifierType.PREDICTION_IS_MORE_SPECIFIC
                    row.expected_tgt = "; ".join(more_general_expected)
                    break
                rows.append(row)
        for expected_tgt in expected_tgts:
            # any remaining expected targets that are unaccounted for are false negatives
            if expected_tgt in expected_accounted_for:
                continue
            row = RelationshipEval(term_id=term_id,
                                   term_label=adapter.label(term_id),
                                   pred=pred,
                                   expected_tgt=expected_tgt,
                                   outcome=OutcomeType.FALSE_NEGATIVE)
            rows.append(row)
    term_label = adapter.label(term_id)
    for row in rows:
        row.term_label=term_label
        row.ontology = ont
    return rows
                
rows = get_relationship_outcomes(
    {
         "subClassOf": ["Cell"],
         "PartOf": ["Cerebellum"], 
         "DevelopsFrom": ["StemCell"],
         "OnlyInTaxon": ["HomoSapiens"]
    }, 
    {
        "subClassOf": ["Neuron"], # more specific
        "PartOf": ["Brain", "MadeUp"], # more general, new
        "MadeUpRel": ["z", "y"], 
        "OnlyInTaxon": ["HomoSapiens"]
    }, 
    "cl",
    term_id="CL:0000540"
)
print(len(rows))
for row in rows:
    print(row.model_dump())
    
df = pd.DataFrame([row.as_dict() for row in rows])
df

INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Number of nodes in background set=16180
INFO:root:Number of nodes in background set=16180


MISSING: MadeUpRel
5
{'ontology': 'cl', 'method': None, 'term_id': 'CL:0000540', 'term_label': 'neuron', 'pred': 'DevelopsFrom', 'outcome': <OutcomeType.FALSE_NEGATIVE: 'false_negative'>, 'qualifier': None, 'expected_tgt': 'StemCell', 'predicted_tgt': None, 'predicted_tgt_exists': True, 'ic_ratio': None}
{'ontology': 'cl', 'method': None, 'term_id': 'CL:0000540', 'term_label': 'neuron', 'pred': 'subClassOf', 'outcome': <OutcomeType.FALSE_NEGATIVE: 'false_negative'>, 'qualifier': None, 'expected_tgt': 'Cell', 'predicted_tgt': None, 'predicted_tgt_exists': True, 'ic_ratio': None}
{'ontology': 'cl', 'method': None, 'term_id': 'CL:0000540', 'term_label': 'neuron', 'pred': 'PartOf', 'outcome': <OutcomeType.FALSE_NEGATIVE: 'false_negative'>, 'qualifier': <QualifierType.PREDICTION_IS_MORE_GENERAL: 'prediction_is_more_general'>, 'expected_tgt': 'Cerebellum', 'predicted_tgt': 'Brain', 'predicted_tgt_exists': True, 'ic_ratio': 0.5334318196772811}
{'ontology': 'cl', 'method': None, 'term_id': 'CL

,ontology,method,term_id,term_label,pred,outcome,qualifier,expected_tgt,predicted_tgt,predicted_tgt_exists,ic_ratio
0,cl,None,CL:0000540,neuron,DevelopsFrom,false_negative,None,StemCell,None,True,NaN
1,cl,None,CL:0000540,neuron,subClassOf,false_negative,None,Cell,None,True,NaN
2,cl,None,CL:0000540,neuron,PartOf,false_negative,prediction_is_more_general,Cerebellum,Brain,True,0.533432
3,cl,None,CL:0000540,neuron,PartOf,false_positive,None,None,MadeUp,False,NaN
4,cl,None,CL:0000540,neuron,OnlyInTaxon,true_positive,None,HomoSapiens,HomoSapiens,True,NaN


In [80]:
def parse_relationships(rel_str: str) -> Dict[str, List[str]]:
    """
    unflatten results
    """
    if pd.isnull(rel_str):
        return {}
    if not isinstance(rel_str, str):
        print(f"BAD: {rel_str}")
        return {}
    rels = yaml.safe_load(rel_str)
    m = defaultdict(list)
    for rel in rels:
        if "predicate" not in rel or "target" not in rel:
            print(f"BAD REL: {rel} in {rel_str}")
            continue
        if not isinstance(rel["target"], str):
            print(f"BAD REL: {rel} in {rel_str}")
            continue
        m[rel["predicate"]].append(rel["target"])
    return m

In [81]:
def parse_predictions_to_outcomes(f: str, ont: str, preds: List=None, strict=False, method="no_method") -> List[dict]:
    #adapter = get_adapter(f"sqlite:obo:{ont}")
    res = pd.read_csv(f, comment="#", sep="\t")
    rows = []
    for _, row in res.iterrows():
        expected_rels = parse_relationships(row['expected_relationships'])
        predicted_rels = parse_relationships(row['predicted_relationships'])
        term_id = row["masked_original_id"]
        if preds:
            expected_rels = {k: v for k, v in expected_rels.items() if k in preds}
            predicted_rels = {k: v for k, v in predicted_rels.items() if k in preds}
            base = f"{method}-{'+'.join(preds)}"
        else:
            base = method
        #print(f"Scoring: {expected_rels} // {predicted_rels}")
        this_rows = get_relationship_outcomes(expected_rels, predicted_rels, ont, term_id=term_id)
        rows.extend(this_rows)
    for row in rows:
        row.method = method
    return [row.as_dict() for row in rows]

rows = parse_predictions_to_outcomes("results/ont_cl-Prelationships-Mid.original_id-BGFalse-TrNone-Te50-Mgpt-4-EMopenai.results.tsv", "cl", method="RAG+gpt4")
df = pd.DataFrame(rows)
df.to_csv("foo.tsv", sep="\t")
df


INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.d

MISSING: ReceivesConnectivityInputFrom
MISSING: AxonIn


INFO:root:Number of nodes in background set=16180
INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Number of nodes in background set=16180
INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.

MISSING: ConnectsTo


INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Number of nodes in background set=16180
INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-p

...

MISSING: isPartOf
MISSING: SynapsesWith
MISSING: minimalContactWith


INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Number of nodes in background set=16180


MISSING: LacksIntracellularComponent


INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db


,ontology,method,term_id,term_label,pred,outcome,qualifier,expected_tgt,predicted_tgt,predicted_tgt_exists,ic_ratio
0,cl,RAG+gpt4,CL:4033045,lung migratory dendritic cell,PartOf,false_positive,None,None,Lung,True,NaN
1,cl,RAG+gpt4,CL:4033045,lung migratory dendritic cell,subClassOf,true_positive,None,DendriticCell,DendriticCell,True,NaN
2,cl,RAG+gpt4,CL:4033019,ON-blue cone bipolar cell,subClassOf,false_positive,None,None,PhotopicRetinalBipolarCell,False,NaN
3,cl,RAG+gpt4,CL:4033019,ON-blue cone bipolar cell,subClassOf,false_negative,None,ONBipolarCell,None,True,NaN
4,cl,RAG+gpt4,CL:4033029,diffuse bipolar 3a cell,subClassOf,false_positive,None,None,OFFCalbindinPositiveBipolarCell,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
157,cl,RAG+gpt4,CL:4030037,late spermatid,CapableOf,false_positive,None,None,SpermRelease,False,NaN
158,cl,RAG+gpt4,CL:4030037,late spermatid,CapableOf,false_negative,None,SpermatidDevelopment,None,True,NaN
159,cl,RAG+gpt4,CL:4030037,late spermatid,subClassOf,true_positive,None,Spermatid,Spermatid,True,NaN
160,cl,RAG+gpt4,CL:4030035,dental pulp stem cell,subClassOf,true_positive,None,StemCell,StemCell,True,NaN


In [63]:
def get_method(obj: dict) -> str:
    """
    gets the method from the yaml object
    """
    if obj.get("generate_background"):
        return "RAG+background"
    additional = obj.get("additional_collections", [])
    if additional:
        assert len(additional) == 1
        x = additional[0]
        if x.startswith("gh_"):
            return "RAG+github"
        elif x.startswith("devdocs"):
            return "RAG+devdocs"
        else:
            assert False
    else:
        return "RAG"

In [68]:
import re

def load_all_results(preds=None, glob_expr="results/*.results.yaml"):
    files = glob.glob(glob_expr)
    rows = []
    for f in files:
        if "Prelationships" not in f:
            continue
        print(f)
        obj = yaml.safe_load(open(f))
        # look for string '/ont_{ONT}-Prelationships' using regex
        m = re.search(r"ont_(\w+)-Prelationships", f)
        assert m
        meth = get_method(obj) + "-" + obj["model_name"]
        print(f, meth)
        ont = m.group(1)
        this_rows = parse_predictions_to_outcomes(f.replace(".yaml", ".tsv"), ont, preds, strict=False, method=meth)
        #for row in this_rows:
        #    row = {**obj, **row}
        #    # row["method"] = meth
        #    rows.append(row)
        rows.extend(this_rows)
        
    return pd.DataFrame(rows)


In [70]:
#df = load_all_results(glob_expr="results/ont_cl-Prelationships-Mid.original_id-BGFalse-TrNone-Te50-Mgpt-4-EMopenai.results.yaml")
#df

INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.d

results/ont_cl-Prelationships-Mid.original_id-BGFalse-TrNone-Te50-Mgpt-4-EMopenai.results.yaml
results/ont_cl-Prelationships-Mid.original_id-BGFalse-TrNone-Te50-Mgpt-4-EMopenai.results.yaml RAG-gpt-4
MISSING: ReceivesConnectivityInputFrom
MISSING: AxonIn


INFO:root:Number of nodes in background set=16180
INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Number of nodes in background set=16180
INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.

MISSING: ConnectsTo


INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Number of nodes in background set=16180
INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-p

...

MISSING: isPartOf
MISSING: SynapsesWith
MISSING: minimalContactWith


INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Number of nodes in background set=16180


MISSING: LacksIntracellularComponent


INFO:root:Number of nodes in background set=16180
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db
INFO:root:Locator: obo:cl
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/cl.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/cl.db


,ontology,method,term_id,term_label,pred,outcome,qualifier,expected_tgt,predicted_tgt,predicted_tgt_exists,ic_ratio
0,cl,RAG-gpt-4,CL:4033045,lung migratory dendritic cell,PartOf,false_positive,None,None,Lung,True,NaN
1,cl,RAG-gpt-4,CL:4033045,lung migratory dendritic cell,subClassOf,true_positive,None,DendriticCell,DendriticCell,True,NaN
2,cl,RAG-gpt-4,CL:4033019,ON-blue cone bipolar cell,subClassOf,false_positive,None,None,PhotopicRetinalBipolarCell,False,NaN
3,cl,RAG-gpt-4,CL:4033019,ON-blue cone bipolar cell,subClassOf,false_negative,None,ONBipolarCell,None,True,NaN
4,cl,RAG-gpt-4,CL:4033029,diffuse bipolar 3a cell,subClassOf,false_positive,None,None,OFFCalbindinPositiveBipolarCell,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
157,cl,RAG-gpt-4,CL:4030037,late spermatid,CapableOf,false_positive,None,None,SpermRelease,False,NaN
158,cl,RAG-gpt-4,CL:4030037,late spermatid,CapableOf,false_negative,None,SpermatidDevelopment,None,True,NaN
159,cl,RAG-gpt-4,CL:4030037,late spermatid,subClassOf,true_positive,None,Spermatid,Spermatid,True,NaN
160,cl,RAG-gpt-4,CL:4030035,dental pulp stem cell,subClassOf,true_positive,None,StemCell,StemCell,True,NaN


In [82]:
df = load_all_results()
df

INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.d

results/ont_go-Prelationships-Mid.original_id-BGTrue-TrNone-Te50-Mgpt-4-EMopenai.results.yaml
results/ont_go-Prelationships-Mid.original_id-BGTrue-TrNone-Te50-Mgpt-4-EMopenai.results.yaml RAG+background-gpt-4


INFO:root:Number of nodes in background set=83855
INFO:root:Number of nodes in background set=83855
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Number of nodes in background set=83855
INFO:root:Number of nodes in background set=83855
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.

MISSING: HasProduct


INFO:root:Number of nodes in background set=83855
INFO:root:Number of nodes in background set=83855
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Number of nodes in background set=83855
INFO:root:Number of nodes in background set=83855
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/go.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/go.db
INFO:root:Locator: obo:go
INFO:root:Ensuring gunzipped for https://s3.amazonaws.

...

INFO:root:Number of nodes in background set=37749
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/foodon.db
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/foodon.db
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/foodon.db
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/foodon.db
INFO:root:Number of nodes in background set=37749


MISSING: HasQuality_RO:0000086


INFO:root:Number of nodes in background set=37749
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/foodon.db
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/foodon.db
INFO:root:Number of nodes in background set=37749
INFO:root:Number of nodes in background set=37749
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data/oaklib/foodon.db
INFO:root:Number of nodes in background set=37749
INFO:root:Number of nodes in background set=37749
INFO:root:Locator: obo:foodon
INFO:root:Ensuring gunzipped for https://s3.amazonaws.com/bbop-sqlite/foodon.db.gz
INFO:root:Locator, post-processed: sqlite:////Users/cjm/.data

MISSING: HasQuality_RO:0000086


,ontology,method,term_id,term_label,pred,outcome,qualifier,expected_tgt,predicted_tgt,predicted_tgt_exists,ic_ratio
0,go,RAG+background-gpt-4,GO:0160073,Casparian strip assembly,PartOf,false_negative,None,PlantTypeCellWallAssembly,None,True,NaN
1,go,RAG+background-gpt-4,GO:0160073,Casparian strip assembly,ResultsInAssemblyOf,true_positive,None,CasparianStrip,CasparianStrip,True,NaN
2,go,RAG+background-gpt-4,GO:0160073,Casparian strip assembly,subClassOf,true_positive,None,CellularComponentAssembly,CellularComponentAssembly,True,NaN
3,go,RAG+background-gpt-4,GO:0120340,radial spoke base 1,PartOf,true_positive,None,RadialSpoke1,RadialSpoke1,True,NaN
4,go,RAG+background-gpt-4,GO:0120340,radial spoke base 1,subClassOf,true_positive,None,RadialSpokeBase,RadialSpokeBase,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5298,foodon,RAG+background-gpt-4,FOODON:00004454,green cardamom seed (dried),DerivesFrom,false_positive,None,None,GreenCardamomPlant,True,NaN
5299,foodon,RAG+background-gpt-4,FOODON:00004454,green cardamom seed (dried),subClassOf,true_positive,None,CardamomFoodProduct,CardamomFoodProduct,True,NaN
5300,foodon,RAG+background-gpt-4,FOODON:00004438,common millet (cooked),subClassOf,false_positive,None,None,Food_cooked_,False,NaN
5301,foodon,RAG+background-gpt-4,FOODON:00004438,common millet (cooked),subClassOf,false_positive,None,None,Common_millet_wholeOrPieces_,False,NaN


In [83]:
df

,ontology,method,term_id,term_label,pred,outcome,qualifier,expected_tgt,predicted_tgt,predicted_tgt_exists,ic_ratio
0,go,RAG+background-gpt-4,GO:0160073,Casparian strip assembly,PartOf,false_negative,None,PlantTypeCellWallAssembly,None,True,NaN
1,go,RAG+background-gpt-4,GO:0160073,Casparian strip assembly,ResultsInAssemblyOf,true_positive,None,CasparianStrip,CasparianStrip,True,NaN
2,go,RAG+background-gpt-4,GO:0160073,Casparian strip assembly,subClassOf,true_positive,None,CellularComponentAssembly,CellularComponentAssembly,True,NaN
3,go,RAG+background-gpt-4,GO:0120340,radial spoke base 1,PartOf,true_positive,None,RadialSpoke1,RadialSpoke1,True,NaN
4,go,RAG+background-gpt-4,GO:0120340,radial spoke base 1,subClassOf,true_positive,None,RadialSpokeBase,RadialSpokeBase,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5298,foodon,RAG+background-gpt-4,FOODON:00004454,green cardamom seed (dried),DerivesFrom,false_positive,None,None,GreenCardamomPlant,True,NaN
5299,foodon,RAG+background-gpt-4,FOODON:00004454,green cardamom seed (dried),subClassOf,true_positive,None,CardamomFoodProduct,CardamomFoodProduct,True,NaN
5300,foodon,RAG+background-gpt-4,FOODON:00004438,common millet (cooked),subClassOf,false_positive,None,None,Food_cooked_,False,NaN
5301,foodon,RAG+background-gpt-4,FOODON:00004438,common millet (cooked),subClassOf,false_positive,None,None,Common_millet_wholeOrPieces_,False,NaN


In [84]:
df.to_csv("results/predicted-relationships-for-eval.tsv", sep="\t", index=False)

In [85]:
df.query("method == 'RAG-gpt-4'")

,ontology,method,term_id,term_label,pred,outcome,qualifier,expected_tgt,predicted_tgt,predicted_tgt_exists,ic_ratio
137,hp,RAG-gpt-4,HP:0034745,Elevated circulating S-sulfocysteine concentra...,subClassOf,false_positive,None,None,AbnormalCirculatingSulfurAminoAcidConcentration,True,NaN
138,hp,RAG-gpt-4,HP:0034745,Elevated circulating S-sulfocysteine concentra...,subClassOf,false_negative,None,AbnormalCirculatingNonProteinogenicAminoAcidCo...,None,True,NaN
139,hp,RAG-gpt-4,HP:0034604,Elevated circulating DOPA concentration,subClassOf,false_positive,None,None,AbnormalCirculatingAminoAcidConcentration,True,NaN
140,hp,RAG-gpt-4,HP:0034604,Elevated circulating DOPA concentration,subClassOf,false_negative,None,AbnormalCirculatingMonocarboxylicAcidConcentra...,None,True,NaN
141,hp,RAG-gpt-4,HP:0034566,Aplasia of posterior communicating artery,subClassOf,true_positive,None,AbnormalPosteriorCirculatingArteryMorphology,AbnormalPosteriorCirculatingArteryMorphology,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4976,foodon,RAG-gpt-4,FOODON:00004454,green cardamom seed (dried),DerivesFrom,false_positive,None,None,GreenCardamomPlant,True,NaN
4977,foodon,RAG-gpt-4,FOODON:00004454,green cardamom seed (dried),subClassOf,true_positive,None,CardamomFoodProduct,CardamomFoodProduct,True,NaN
4978,foodon,RAG-gpt-4,FOODON:00004438,common millet (cooked),subClassOf,false_positive,None,None,Millet_cooked_,False,NaN
4979,foodon,RAG-gpt-4,FOODON:00004438,common millet (cooked),subClassOf,false_positive,None,None,Food_cooked_,False,NaN


In [73]:
df.query("method == 'RAG-gpt-4'").to_csv("results/predicted-relationships-for-eval-rag-gpt-4.tsv", sep="\t", index=False)

## Logical Definitions

In [ ]:
import numpy as np

def load_ldef_results_tsv(f):
    res = pd.read_csv(f, comment="#", sep="\t")
    res['expected_logical_definition'].replace('', np.nan, inplace=True)
    res = res[res['expected_logical_definition'].notnull()]
    # Calculate Accuracy per row: (TP + TN) / (TP + TN + FP + FN)
    # Since we can't calculate TN here, we'll omit it from the formula
    res['accuracy'] = res['metric_true_positives'] / (res['metric_true_positives'] + res['metric_false_positives'] + res['metric_false_negatives'])
    res['precision'] = res['metric_true_positives'] / (res['metric_true_positives'] + res['metric_false_positives'])

    return res
    

res = load_ldef_results_tsv("results/ont_go-Plogical_definition-Mid.original_id-BGFalse-TrNone-Te50-Mgpt-4-EMopenai.results.tsv")
res.describe(include="all")

In [ ]:
def load_all_results_ldefs():
    files = glob.glob("results/*.results.yaml")
    objs = []
    for f in files:
        obj = load_results(f)
        if obj["task"] != "logical_definition":
            continue
        accuracy = load_ldef_results_tsv(f.replace(".yaml", ".tsv"))["accuracy"].mean()
        obj["accuracy"] = accuracy
        objs.append(obj)
    return objs

objs = load_all_results_ldefs()

In [ ]:
ldefs_df = pd.DataFrame(objs)

### Logical Definitions (Excluding terms with none)

In [ ]:
hm("logical_definition", "direct", ldefs_df)

### Logical Definitions (including over-prediction)

In [ ]:
hm("logical_definition", "direct")

## Definitions Task

### Definitions (direct)

No additional background knowledge

In [ ]:
hm("definition", "direct")

In [ ]:
my_ttest("definition", "direct")

### Definitions with github as background method

In [ ]:
hm("definition", "github")

### Definitions with auto-generated background summaries

In [ ]:
hm("definition", "background")

In [ ]:
lp("definition")

### Definitions using developer docs

In [ ]:
df.query("ontology == 'cl' and task == 'definition'")["method"].unique()

In [ ]:
hm("definition", "devdocs", df.query("ontology == 'cl'"))

### Significance of different methods

In [ ]:
methods = all_df["method"].unique()
models = all_df["model_name"].unique()
for m1 in methods:
    for m2 in methods:
        if m2 >= m1:
            continue
        for mn in list(models) + [None]:
            if mn is None:
                all_df_x = all_df.query("task == 'definition'")
            else:
                all_df_x = all_df.query(f"task == 'definition' and model_name == '{mn}'")
            m1_data = all_df_x[(all_df_x['method'] == m1)]['accuracy']
            m2_data = all_df_x[(all_df_x['method'] == m2)]['accuracy']

            # Step 2: Conduct the T-Test
            # Use ttest_ind to conduct the t-test
            t_statistic, p_value = stats.ttest_ind(m1_data, m2_data)

            print(f'{mn}:: {m1} v {m2}: T-statistic: {t_statistic}')
            print(f'{mn}:: {m1} v {m2}: P-value: {p_value}')


## Subsumption Analysis

Note that the relationships stats above are calculated on predicting *all* relationships.

To compare with owl2vec, we filter these for only subsumptions (subClassOf) relationships. See https://arxiv.org/pdf/2009.14654.pdf


In [ ]:
import json

def subsumptions(rel_str: str):
    if pd.isnull(rel_str):
        return []
    if not isinstance(rel_str, str):
        print(f"BAD: {rel_str}")
        return []
    rels = yaml.safe_load(rel_str)
    for rel in rels:
         if "predicate" not in rel:
             print(f"BAD REL: {rel} in {rel_str}")
    return [r["target"] for r in rels if r.get("predicate", "") =="subClassOf"]

def calculate_metrics(row):
    expected_set = set(row['expected_subsumptions'])
    predicted_set = set(row['predicted_subsumptions'])
    
    TP = len(expected_set & predicted_set)
    FP = len(predicted_set - expected_set)
    FN = len(expected_set - predicted_set)
    
    # In a classical setting, TN would be calculated here. 
    # However, if you don't have information about what could have been predicted 
    # but wasn't (i.e., the universal set), TN can't be calculated.
    
    return pd.Series({'TP': TP, 'FP': FP, 'FN': FN})
    

def load_subsumption_results_tsv(f):
    res = pd.read_csv(f, comment="#", sep="\t")
    res['expected_subsumptions'] = res.apply(lambda row : subsumptions(row['expected_relationships']), axis = 1)
    res['predicted_subsumptions'] = res.apply(lambda row : subsumptions(row['predicted_relationships']), axis = 1)
    metrics_df = res.apply(calculate_metrics, axis=1)

    # Add the metrics as new columns to the original DataFrame
    df_with_metrics = pd.concat([df, metrics_df], axis=1)

    # Calculate Accuracy per row: (TP + TN) / (TP + TN + FP + FN)
    # Since we can't calculate TN here, we'll omit it from the formula
    df_with_metrics['accuracy'] = df_with_metrics['TP'] / (df_with_metrics['TP'] + df_with_metrics['FP'] + df_with_metrics['FN'])
    df_with_metrics['precision'] = df_with_metrics['TP'] / (df_with_metrics['TP'] + df_with_metrics['FP'])

    return df_with_metrics
    

res = load_subsumption_results_tsv("results/ont_uberon-Prelationships-Mid.original_id-BGFalse-TrNone-Te40-Mgpt-4-EMopenai.results.tsv")
res.describe(include="all")

In [ ]:
import json

def subsumptions(rel_str: str):
    if pd.isnull(rel_str):
        return []
    if not isinstance(rel_str, str):
        print(f"BAD: {rel_str}")
        return []
    rels = yaml.safe_load(rel_str)
    for rel in rels:
         if "predicate" not in rel:
             print(f"BAD REL: {rel} in {rel_str}")
    return [r["target"] for r in rels if r.get("predicate", "") =="subClassOf"]

def calculate_metrics(row):
    expected_set = set(row['expected_subsumptions'])
    predicted_set = set(row['predicted_subsumptions'])
    
    TP = len(expected_set & predicted_set)
    FP = len(predicted_set - expected_set)
    FN = len(expected_set - predicted_set)
    
    # In a classical setting, TN would be calculated here. 
    # However, if you don't have information about what could have been predicted 
    # but wasn't (i.e., the universal set), TN can't be calculated.
    
    return pd.Series({'TP': TP, 'FP': FP, 'FN': FN})
    

def load_subsumption_results_tsv(f):
    res = pd.read_csv(f, comment="#", sep="\t")
    res['expected_subsumptions'] = res.apply(lambda row : subsumptions(row['expected_relationships']), axis = 1)
    res['predicted_subsumptions'] = res.apply(lambda row : subsumptions(row['predicted_relationships']), axis = 1)
    metrics_df = res.apply(calculate_metrics, axis=1)

    # Add the metrics as new columns to the original DataFrame
    df_with_metrics = pd.concat([df, metrics_df], axis=1)

    # Calculate Accuracy per row: (TP + TN) / (TP + TN + FP + FN)
    # Since we can't calculate TN here, we'll omit it from the formula
    df_with_metrics['accuracy'] = df_with_metrics['TP'] / (df_with_metrics['TP'] + df_with_metrics['FP'] + df_with_metrics['FN'])
    df_with_metrics['precision'] = df_with_metrics['TP'] / (df_with_metrics['TP'] + df_with_metrics['FP'])

    return df_with_metrics
    

#res = load_subsumption_results_tsv("results/ont_uberon-Prelationships-Mid.original_id-BGFalse-TrNone-Te40-Mgpt-4-EMopenai.results.tsv")
#res.describe(include="all")

In [ ]:
res["accuracy"].mean()

In [ ]:
def load_all_results_subsumptions():
    files = glob.glob("results/*.results.yaml")
    objs = []
    for f in files:
        obj = load_results(f)
        if obj["task"] != "relationships":
            continue
        accuracy = load_subsumption_results_tsv(f.replace(".yaml", ".tsv"))["accuracy"].mean()
        obj["accuracy"] = accuracy
        objs.append(obj)
    return objs

objs = load_all_results_subsumptions()

### Main subsumption results

In [ ]:
subs_df = pd.DataFrame(objs)

In [ ]:
hm("relationships", "direct", subs_df)

In [ ]:
hm("relationships", "direct", subs_df, metric="precision")

### Subsumptions using background

In [31]:
hm("relationships", "background", subs_df)

NameError: name 'hm' is not defined

### Subsumptions using github

In [32]:
#hm("relationships", "github", subs_df)

## Preparing definitions for evaluation

In [ ]:
defs_df = all_df.query("task == 'definition'")
defs_df.groupby('ontology').size().reset_index(name='count')

In [ ]:
len(defs_df)

In [ ]:
len(defs_df["expected_definition"].unique())

In [ ]:

df_defs_gpt_direct = pd.read_csv("results/cl-defs-direct.tsv", sep="\t")
gpt_direct = { row["id"]: row["definition"] for _, row in df_defs_gpt_direct.iterrows() }
objs = []
for _, row in defs_df.iterrows():
    id = row["masked_original_id"]
    if id not in gpt_direct:
        continue
    row = {**row}
    row["predicted_definition"] = gpt_direct[id]
    row["model_name"] = "gpt-4"
    row["method"] = "no_RAG"
    del gpt_direct[id]
    objs.append(row)
all_defs_df = pd.concat([defs_df, pd.DataFrame(objs)])
all_defs_df